In [1]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm

# from timelab import MultiColumn, from_data, from_xy_data
# from timelab.utils import smash_array
import sys
sys.path.append('../')
from wavy import *

In [2]:
df = pd.read_pickle('processed.pkl')

In [3]:
# df = df[['LNC', 'MAS', 'CSX']]
df = df[['MAS']]

In [4]:
df.head(5)

MAS                                            
                 Open       High        Low      Close     Volume
Date                                                             
2005-12-21  18.991695  19.154659  18.903944  19.085712  1384200.0
2005-12-22  19.085703  19.298810  18.910202  18.929007  1576900.0
2005-12-23  18.954090  18.954090  18.684571  18.816196  2090500.0
2005-12-27  18.929008  18.991687  18.803650  18.847525  1609000.0
2005-12-28  18.853798  18.935281  18.747245  18.803656  2026300.0

In [5]:
panel = from_data(df, lookback=10, horizon=1)

In [6]:
panel

size                            3737
lookback                          10
horizon                            1
gap                                0
num_xassets                        1
num_yassets                        1
num_xchannels                      5
num_ychannels                      5
start            2005-12-21 00:00:00
end              2020-11-10 00:00:00
Name: Panel, dtype: object


<Panel, size 3737>

In [7]:
a = panel[0:5]

In [9]:
a

size                               5
lookback                          10
horizon                            1
gap                                0
num_xassets                        1
num_yassets                        1
num_xchannels                      5
num_ychannels                      5
start            2005-12-21 00:00:00
end              2006-01-12 00:00:00
Name: Panel, dtype: object


<Panel, size 5>

In [ ]:
Panel(Side(panel.x[0:2]), Side(panel.y[0:2]))

In [ ]:
type(a[0])

In [ ]:
panel[2:]

In [10]:
panel[0]

size                               1
lookback                          10
horizon                            1
gap                                0
num_xassets                        1
num_yassets                        1
num_xchannels                      5
num_ychannels                      5
start            2005-12-21 00:00:00
end              2006-01-06 00:00:00
Name: Panel, dtype: object


<Panel, size 1>

In [ ]:
a = panel.x.pct_change()

In [ ]:
a[0]

In [ ]:
panel.x = panel.x.iloc[1:]

In [ ]:
panel

In [ ]:
type(panel.x)

In [ ]:
panel.x = panel.x.shift(2)

In [ ]:
panel.x.first

In [ ]:
panel.x.tensor3d[1]

In [ ]:
panel.x.dropna()

In [ ]:
type(panel.x[0])

In [ ]:
a = getattr(Block, 'iloc')

In [ ]:
a

In [ ]:
a = panel.x.iloc[:2]

In [ ]:
a.first

In [ ]:
getattr(panel.x.first, a[0].__name__)

In [ ]:
panel.x.first.iloc[1:]

In [ ]:
panel.x = panel.x.fillna(0)
# WHAT? I commented this. Please take a look at panel.py >> 197

In [ ]:
panel.x = panel.x.fillna(0)

In [ ]:
panel.x[0]

In [ ]:
panel.y[0]

In [ ]:
panel.y = panel.y.filter(channels='High')
# Too slow. Just imagine an initial df of 1M rows.
# Let's time each of the panel/side functions and see try to speed up the slow ones.

In [ ]:
panel.y[0]

In [ ]:
panel.y.values
# ???

In [ ]:
baseline = Baseline(panel)

In [ ]:
y_true = panel.test.y.flatten()
y_pred = baseline.predict()

In [ ]:
y_pred = y_pred.predicted

In [ ]:
y_true.shape

In [ ]:
y_true[:3]

In [ ]:
y_pred[:3]

In [ ]:
# Maybe the predictions should come as series/dataframes indeed. Here, the INDEX is really important, and we could remove the first NaN as long as we keep the index of the second value explicit.

# Let's think of a way to make indexes intuitive. They are simple when input index is dates, but when this is not passed, just numeric indexes would work?

# This means quite a shift in the way we treat a panel's/side index. It should be fixed in a way that even when dropping samples or adding steps, etc. the index is still consistent.

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_true, y_pred)
# NaN makes it not computable.

In [ ]:
mean_squared_error(y_true[1:], y_pred[1:]) # horizon = 1, thus that's the number of nans generated with baseline model.

# If the index is explicit and we use dataframes, we could hide NaN values...
# metrics such as mean_squared_error could be part of the Model, e.g. model.validate('mse'), model.test('mse)

In [ ]:
panel.x.findna()